In [72]:
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import random
import re

%matplotlib inline

os.listdir()

## Gather

In [3]:
edf = pd.read_csv('all_energy_statistics.csv')

## Assess 1

In [4]:
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189482 entries, 0 to 1189481
Data columns (total 7 columns):
country_or_area          1189482 non-null object
commodity_transaction    1189482 non-null object
year                     1189482 non-null int64
unit                     1189482 non-null object
quantity                 1189482 non-null float64
quantity_footnotes       163946 non-null float64
category                 1189482 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 63.5+ MB


In [5]:
edf.sample(5)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
476255,Jersey,Gas Oil/ Diesel Oil - Transformation in electr...,1997,"Metric tons, thousand",15.379,NaN,gas_oil_diesel_oil
662111,Nicaragua,Liquefied petroleum gas (LPG) - production fro...,2007,"Metric tons, thousand",16.000,NaN,liquified_petroleum_gas
238141,Isle of Man,Electricity - total net installed capacity of ...,2012,"Kilowatts, thousand",1.000,NaN,electricity_net_installed_capacity_of_electric...
676995,Spain,Liquified Petroleum Gas (LPG) - Consumption by...,2008,"Metric tons, thousand",11.000,NaN,liquified_petroleum_gas
577342,Pakistan,Kerosene-type Jet Fuel - Consumption by domest...,1999,"Metric tons, thousand",641.000,NaN,kerosene_type_jet_fuel


In [6]:
edf.country_or_area.describe()

count     1189482
unique        243
top       Germany
freq        20422
Name: country_or_area, dtype: object

In [7]:
edf.unit.value_counts()

Metric tons,  thousand     759859
Terajoules                 178937
Kilowatt-hours, million    147741
Cubic metres, thousand      52032
Kilowatts,  thousand        50229
Metric Tons                   684
Name: unit, dtype: int64

In [8]:
edf[edf.category.isnull()]

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category


In [9]:
edf[edf.quantity_footnotes.notnull()]

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
622,Colombia,Additives and Oxygenates - Production,2014,"Metric tons, thousand",34.530,1.0,additives_and_oxygenates
1369,Colombia,Additives and Oxygenates - Total energy supply,2014,"Metric tons, thousand",34.530,1.0,additives_and_oxygenates
1886,Colombia,Additives and Oxygenates - transfers and recyc...,2014,"Metric tons, thousand",34.530,1.0,additives_and_oxygenates
2250,Jordan,Additives and Oxygenates - Transformation,2014,"Metric tons, thousand",103.000,1.0,additives_and_oxygenates
2251,Jordan,Additives and Oxygenates - Transformation,2013,"Metric tons, thousand",101.000,1.0,additives_and_oxygenates
2252,Jordan,Additives and Oxygenates - Transformation,2012,"Metric tons, thousand",107.000,1.0,additives_and_oxygenates
2253,Jordan,Additives and Oxygenates - Transformation,2011,"Metric tons, thousand",131.000,1.0,additives_and_oxygenates
2753,Jordan,Additives and Oxygenates - Transformation in o...,2014,"Metric tons, thousand",103.000,1.0,additives_and_oxygenates
2754,Jordan,Additives and Oxygenates - Transformation in o...,2013,"Metric tons, thousand",101.000,1.0,additives_and_oxygenates
2755,Jordan,Additives and Oxygenates - Transformation in o...,2012,"Metric tons, thousand",107.000,1.0,additives_and_oxygenates


In [10]:
edf.category.value_counts()

total_electricity                                              133916
gas_oil_diesel_oil                                              97645
fuel_oil                                                        75132
natural_gas_including_lng                                       64161
liquified_petroleum_gas                                         62156
motor_gasoline                                                  53198
fuelwood                                                        52032
electricity_net_installed_capacity_of_electric_power_plants     50229
other_kerosene                                                  43466
hard_coal                                                       42307
kerosene_type_jet_fuel                                          34558
heat                                                            29656
charcoal                                                        28000
conventional_crude_oil                                          27352
other_bituminous_coa

In [11]:
edf.commodity_transaction.value_counts()

From combustible fuels – Main activity                                                               6601
Electricity - Gross demand                                                                           5532
Electricity - Gross production                                                                       5523
Electricity - total production, main activity                                                        5523
Electricity - net production                                                                         5523
Electricity - Own use by electricity, heat and CHP plants                                            5523
Electricity - total net installed capacity of electric power plants, main activity                   5521
Electricity - total net installed capacity of electric power plants, main activity & autoproducer    5521
Electricity - Final energy consumption                                                               5499
Electricity - Consumption by other            

In [12]:
edf.commodity_transaction.unique()

array(['Additives and Oxygenates - Exports',
       'Additives and Oxygenates - Imports',
       'Additives and Oxygenates - Production', ...,
       'White spirit and special boiling point industrial spirits - Transformation',
       'White spirit and special boiling point industrial spirits - Transformation in petrochemical plants',
       'Electricity - total wind production'], dtype=object)

## Clean

In [13]:
edf_copy = edf.copy()

### define and code
* split column commodity_transaction on ' - ' using str.split(); return a dataframe, concatenate this data frame with the existing one

In [14]:
edf_split = edf_copy.commodity_transaction.str.split(pat=r'\s-\s', n=0, expand=True)

In [15]:
edf_split.columns = ['commodity', 'transaction', 'modifier']

In [16]:
edf_copy = pd.concat([edf_copy, edf_split], axis=1)

### Test

In [17]:
edf_copy.sample(10)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,commodity,transaction,modifier
1047522,India,Electricity - Consumption by households,2002,"Kilowatt-hours, million",83355.000000,NaN,total_electricity,Electricity,Consumption by households,None
1010393,Russian Federation,Refinery Gas - Consumption by commerce and pub...,2008,"Metric tons, thousand",20.000000,NaN,refinery_gas,Refinery Gas,Consumption by commerce and public services,None
343723,Honduras,Fuelwood - Consumption by other,1994,"Cubic metres, thousand",8022.000000,1.0,fuelwood,Fuelwood,Consumption by other,None
313084,Venezuela (Bolivar. Rep.),Fuel oil - production from refineries,2000,"Metric tons, thousand",13938.271605,NaN,fuel_oil,Fuel oil,production from refineries,None
302973,Israel,Fuel oil - International marine bunkers,1999,"Metric tons, thousand",93.000000,NaN,fuel_oil,Fuel oil,International marine bunkers,None
266779,Mexico,"Fuel oil - Consumption by manufacturing, const...",1993,"Metric tons, thousand",5177.000000,NaN,fuel_oil,Fuel oil,"Consumption by manufacturing, construction and...",None
619631,Georgia,Lignite - Total energy supply,2009,"Metric tons, thousand",168.000000,NaN,lignite,Lignite,Total energy supply,None
968788,India,"Vegetal waste - Transformation in electricity,...",2011,Terajoules,189684.000000,1.0,other_vegetal_material_and_residues,Vegetal waste,"Transformation in electricity, CHP and heat pl...",None
272103,Pakistan,Fuel oil - Consumption by other,2010,"Metric tons, thousand",4.013000,NaN,fuel_oil,Fuel oil,Consumption by other,None
1129915,Zambia,"Electricity - Own use by electricity, heat and...",2007,"Kilowatt-hours, million",237.000000,NaN,total_electricity,Electricity,"Own use by electricity, heat and CHP plants",None


In [18]:
edf_copy[edf_copy.commodity.isnull()]

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,commodity,transaction,modifier


In [19]:
edf_copy[edf_copy.transaction.isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41416 entries, 121420 to 1177351
Data columns (total 10 columns):
country_or_area          41416 non-null object
commodity_transaction    41416 non-null object
year                     41416 non-null int64
unit                     41416 non-null object
quantity                 41416 non-null float64
quantity_footnotes       3819 non-null float64
category                 41416 non-null object
commodity                41416 non-null object
transaction              0 non-null object
modifier                 0 non-null object
dtypes: float64(2), int64(1), object(7)
memory usage: 3.5+ MB


In [20]:
edf_copy[edf_copy.transaction.isnull()].sample(10)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,commodity,transaction,modifier
1169001,Lao People's Dem. Rep.,Solar photovoltaic – Autoproducer,2009,"Kilowatt-hours, million",193.3,1.0,total_electricity,Solar photovoltaic – Autoproducer,None,None
1161833,Albania,Hydro – Main activity,1996,"Kilowatt-hours, million",5726.0,NaN,total_electricity,Hydro – Main activity,None,None
1172391,El Salvador,Total refinery output,2005,"Metric tons, thousand",1041.0,NaN,total_refinery_output,Total refinery output,None,None
1174649,Azerbaijan,Total refinery throughput,1994,"Metric tons, thousand",10497.0,NaN,total_refinery_output,Total refinery throughput,None,None
1164651,Russian Federation,Hydro – Main activity,2000,"Kilowatt-hours, million",164412.0,NaN,total_electricity,Hydro – Main activity,None,None
1152817,Mexico,From combustible fuels – Main activity,2002,"Kilowatt-hours, million",161996.0,NaN,total_electricity,From combustible fuels – Main activity,None,None
1176373,Papua New Guinea,Total refinery throughput,2008,"Metric tons, thousand",804.0,NaN,total_refinery_output,Total refinery throughput,None,None
1167579,Switzerland,Of which: Pumped hydro – Main activity,2003,"Kilowatt-hours, million",2029.0,NaN,total_electricity,Of which: Pumped hydro – Main activity,None,None
1159949,Philippines,Geothermal – Main activity,2003,"Kilowatt-hours, million",9822.0,NaN,total_electricity,Geothermal – Main activity,None,None
1164349,Pakistan,Hydro – Main activity,2004,"Kilowatt-hours, million",25671.0,NaN,total_electricity,Hydro – Main activity,None,None


## Assess
* there is an issue with the split method, not splitting on what appear to be ' - ', the probelm rows have None as transaction in edf_copy, for this subest the index object will be used to find them in the original edf dataframe

In [21]:
problems = edf.loc[edf_copy[edf_copy.transaction.isnull()].index]

In [22]:
problems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41416 entries, 121420 to 1177351
Data columns (total 7 columns):
country_or_area          41416 non-null object
commodity_transaction    41416 non-null object
year                     41416 non-null int64
unit                     41416 non-null object
quantity                 41416 non-null float64
quantity_footnotes       3819 non-null float64
category                 41416 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 2.5+ MB


In [23]:
problems.commodity_transaction.value_counts()

From combustible fuels – Main activity                         6601
From combustible fuels – Autoproducer                          4234
Hydro – Main activity                                          3708
Total refinery throughput                                      2893
Total refinery output                                          2890
From combustible fuels – Main activity – Electricity plants    2785
From combustible fuels – Main activity – CHP plants            1878
From combustible fuels – Autoproducer – Electricity plants     1781
From combustible fuels – Autoproducer – CHP plants             1499
Hydro – Autoproducer                                           1331
Wind – Main activity                                           1277
Nuclear – Main activity                                         909
From combustible fuels – Main activity – Heat plants            851
Solar – Main activity                                           696
Geothermal – Main activity                      

In [24]:
problems.sample(10)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
1164018,Mexico,Hydro – Main activity,1999,"Kilowatt-hours, million",32714.0,NaN,total_electricity
1154765,United Arab Emirates,From combustible fuels – Main activity,2006,"Kilowatt-hours, million",66768.0,NaN,total_electricity
1171858,Belgium,Total refinery output,1997,"Metric tons, thousand",36644.0,NaN,total_refinery_output
1150215,Bolivia (Plur. State of),From combustible fuels – Main activity,1992,"Kilowatt-hours, million",1106.0,NaN,total_electricity
1144945,Japan,From combustible fuels – Autoproducer,2010,"Kilowatt-hours, million",141561.0,NaN,total_electricity
1176122,Myanmar,Total refinery throughput,1994,"Metric tons, thousand",879.0,NaN,total_refinery_output
1148226,Côte d'Ivoire,From combustible fuels – Autoproducer – Electr...,2013,"Kilowatt-hours, million",70.0,NaN,total_electricity
534709,Bulgaria,From combustible fuels – Autoproducer – CHP pl...,2014,Terajoules,87.0,NaN,heat
1144153,Dem. Rep. of the Congo,From combustible fuels – Autoproducer,2004,"Kilowatt-hours, million",0.0,1.0,total_electricity
1168949,Japan,Solar photovoltaic – Autoproducer,2009,"Kilowatt-hours, million",2758.0,NaN,total_electricity


In [25]:
ord(problems.loc[1160362].commodity_transaction.split()[1]) == ord('-')

False

In [26]:
ord(problems.loc[1160362].commodity_transaction.split()[1])

8211

In [27]:
ord('-')

45

In [28]:
ord('-')

45

In [29]:
problems.loc[1160362].commodity_transaction.replace(chr(8211), '-').split(r'-')

['Geothermal ', ' Main activity ', ' Electricity plants']

### findings
* the issue is that what appears to be a hyphen in some rows in the commodity_transaction column is in fact an em-dash
* also noticed previously, an em-dash or hyphen is not always preceeded by white space

## Clean

### define and code
* reconstruct the edf_copy data frame from the original

In [30]:
edf_copy = edf.copy()

### define and code
* replace the em-dash with hyphen in the commodity transaction column (this approach seemed more straighforward than trying to match a hyphen or an em-dash in with str.split and a regex)
* split column commodity_transaction on hyphen or dash using str.split(); return a dataframe, concatenate this data frame with the existing one

In [31]:
edf_copy.commodity_transaction = edf_copy.commodity_transaction.str.replace(chr(8211), '-')

In [32]:
edf_split = edf_copy.commodity_transaction.str.split(pat=r'\s*[-]\s', n=0, expand=True)

In [33]:
edf_split.columns = ['commodity', 'transaction', 'modifier']

In [34]:
edf_copy = pd.concat([edf_copy, edf_split], axis=1)

### test

In [35]:
edf_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189482 entries, 0 to 1189481
Data columns (total 10 columns):
country_or_area          1189482 non-null object
commodity_transaction    1189482 non-null object
year                     1189482 non-null int64
unit                     1189482 non-null object
quantity                 1189482 non-null float64
quantity_footnotes       163946 non-null float64
category                 1189482 non-null object
commodity                1189482 non-null object
transaction              1183699 non-null object
modifier                 47472 non-null object
dtypes: float64(2), int64(1), object(7)
memory usage: 90.8+ MB


In [36]:
edf_copy.modifier.value_counts()

main activity producers    17227
autoproducers              15226
Electricity plants          5887
CHP plants                  3974
Heat plants                 1854
Total                       1507
Main activity producers     1235
Autoproducers                562
Name: modifier, dtype: int64

In [37]:
edf_copy.transaction.value_counts()

Total energy supply                                                                  77769
Final energy consumption                                                             60878
Final consumption                                                                    60217
Production                                                                           50897
Imports                                                                              47001
Consumption by other                                                                 37780
Consumption by households                                                            27506
production from refineries                                                           26964
Exports                                                                              26328
Stock changes                                                                        24218
Transformation                                                                       24149

### define and code
* drop the commodity_transaction column

In [38]:
edf_copy.drop(['commodity_transaction'], axis=1, inplace=True)

### test

In [39]:
edf_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189482 entries, 0 to 1189481
Data columns (total 9 columns):
country_or_area       1189482 non-null object
year                  1189482 non-null int64
unit                  1189482 non-null object
quantity              1189482 non-null float64
quantity_footnotes    163946 non-null float64
category              1189482 non-null object
commodity             1189482 non-null object
transaction           1183699 non-null object
modifier              47472 non-null object
dtypes: float64(2), int64(1), object(6)
memory usage: 81.7+ MB


### define and code
* relace the unit strings

In [40]:
edf_copy['unit'] = edf_copy.unit.str.replace('Metric tons,  thousand', 'thousand metric tons')
edf_copy['unit'] = edf_copy.unit.str.replace('Kilowatt-hours, million', 'million kilowatt-hours')
edf_copy['unit'] = edf_copy.unit.str.replace('Cubic metres, thousand', 'thousand cubic meters')
edf_copy['unit'] = edf_copy.unit.str.replace('Kilowatts,  thousand', 'thousand kilowatts')
edf_copy['unit'] = edf_copy.unit.str.replace('Metric Tons', 'metric tons')
edf_copy['unit'] = edf_copy.unit.str.replace('Terajoules', 'terajoules')

### test

In [41]:
edf_copy.unit.value_counts()

thousand metric tons      759859
terajoules                178937
million kilowatt-hours    147741
thousand cubic meters      52032
thousand kilowatts         50229
metric tons                  684
Name: unit, dtype: int64

### define and code
* convert the year column to datetime using pd.to_datetime(series, format), change the month and day to december 31st or all year entries using datetime.replace

In [42]:
edf_copy['year'] = pd.to_datetime(edf_copy['year'], format='%Y')

In [43]:
edf_copy['year'] = edf_copy['year'].apply(lambda dt: dt.replace(day=31, month=12))

### test

In [44]:
edf_copy.dtypes

country_or_area               object
year                  datetime64[ns]
unit                          object
quantity                     float64
quantity_footnotes           float64
category                      object
commodity                     object
transaction                   object
modifier                      object
dtype: object

### define and code
* convert the unit columns data type to category

In [45]:
edf_copy['unit'] = edf_copy['unit'].astype('category')

### test

In [46]:
edf_copy.dtypes

country_or_area               object
year                  datetime64[ns]
unit                        category
quantity                     float64
quantity_footnotes           float64
category                      object
commodity                     object
transaction                   object
modifier                      object
dtype: object

In [47]:
edf_copy.unit.value_counts()

thousand metric tons      759859
terajoules                178937
million kilowatt-hours    147741
thousand cubic meters      52032
thousand kilowatts         50229
metric tons                  684
Name: unit, dtype: int64

## Assess

In [48]:
edf_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189482 entries, 0 to 1189481
Data columns (total 9 columns):
country_or_area       1189482 non-null object
year                  1189482 non-null datetime64[ns]
unit                  1189482 non-null category
quantity              1189482 non-null float64
quantity_footnotes    163946 non-null float64
category              1189482 non-null object
commodity             1189482 non-null object
transaction           1183699 non-null object
modifier              47472 non-null object
dtypes: category(1), datetime64[ns](1), float64(2), object(5)
memory usage: 73.7+ MB


In [50]:
edf_copy.category = edf_copy.category.str.replace('_', ' ')

In [51]:
edf_copy.category = edf_copy.category.str.strip()

In [52]:
edf_copy.commodity = edf_copy.commodity.str.strip().str.lower()

In [53]:
edf_copy.transaction = edf_copy.transaction.str.strip().str.lower()

In [54]:
edf_copy.modifier = edf_copy.modifier.str.strip().str.lower()

### test

In [55]:
edf_copy.head(5)

,country_or_area,year,unit,quantity,quantity_footnotes,category,commodity,transaction,modifier
0,Austria,1996-12-31,thousand metric tons,5.0,NaN,additives and oxygenates,additives and oxygenates,exports,None
1,Austria,1995-12-31,thousand metric tons,17.0,NaN,additives and oxygenates,additives and oxygenates,exports,None
2,Belgium,2014-12-31,thousand metric tons,0.0,NaN,additives and oxygenates,additives and oxygenates,exports,None
3,Belgium,2013-12-31,thousand metric tons,0.0,NaN,additives and oxygenates,additives and oxygenates,exports,None
4,Belgium,2012-12-31,thousand metric tons,35.0,NaN,additives and oxygenates,additives and oxygenates,exports,None


## Assess
* I want to compare the category and commodity columns across the rows

In [56]:
edf_copy[(edf_copy.category != edf_copy.commodity)].shape

(543346, 9)

In [57]:
edf_copy[(edf_copy.category.str.contains('coal'))].category.value_counts()

hard coal                42307
charcoal                 28000
other bituminous coal    23138
brown coal               19729
coking coal               6470
brown coal briquettes     5903
sub bituminous coal       5405
coal tar                  2245
other coal products        105
Name: category, dtype: int64

In [58]:
edf_copy[edf_copy.commodity.str.contains('coal')].commodity.value_counts()

hard coal                40214
charcoal                 28000
other bituminous coal    23138
brown coal               18616
coking coal               6470
brown coal briquettes     5903
sub-bituminous coal       5405
coal tar                  2245
coal                      1198
lignite brown coal        1113
hrad coal                  895
other coal products        105
Name: commodity, dtype: int64

In [59]:
edf_copy[edf_copy.quantity_footnotes.notnull()].country_or_area.describe()

count     163946
unique       242
top       Malawi
freq        2247
Name: country_or_area, dtype: object

In [60]:
edf_copy.commodity.value_counts()

electricity                      165140
gas oil/ diesel oil               97645
fuel oil                          75132
natural gas (including lng)       64161
motor gasoline                    53198
fuelwood                          52032
liquefied petroleum gas (lpg)     49078
other kerosene                    43466
hard coal                         40214
kerosene-type jet fuel            34558
charcoal                          28000
other bituminous coal             23138
other oil products n.e.c.         22081
lubricants                        21374
conventional crude oil            21160
heat                              21116
bitumen                           20743
from combustible fuels            20119
brown coal                        18616
aviation gasoline                 18171
petroleum coke                    16494
naphtha                           15969
refinery gas                      15706
vegetal waste                     13959
liquified petroleum gas (lpg)     13078


### Findings
* there are 163,946 records (1,189,482 total, around 13% of the data) that have non-null quantity foot notes. Refering to the UN data portal (where this data was obtained) it seems that this means that the quantity figure was an estimate. There is no infomation on the confidence of the estimate
* lignite is brown coal, this can be addressed in the commodity column
* hrad coal enteries can be replaced with hard coal

## Clean

### define and code
* relace 'hrad coal in commodity column with 'hard coal' using the str.replace method, and 'lignite brown coal' with 'brown coal' in the same column using the same method

In [61]:
edf_copy.commodity = edf_copy.commodity.str.replace('hrad coal', 'hard coal')
edf_copy.commodity = edf_copy.commodity.str.replace('lignite brown coal', 'brown coal')

### test

In [62]:
edf_copy[edf_copy.commodity.str.contains('coal')].commodity.value_counts()

hard coal                41109
charcoal                 28000
other bituminous coal    23138
brown coal               19729
coking coal               6470
brown coal briquettes     5903
sub-bituminous coal       5405
coal tar                  2245
coal                      1198
other coal products        105
Name: commodity, dtype: int64

### define and code
* drop the quantity foot notes column

In [63]:
del edf_copy['quantity_footnotes']

### test

In [64]:
edf_copy.columns

Index(['country_or_area', 'year', 'unit', 'quantity', 'category', 'commodity',
       'transaction', 'modifier'],
      dtype='object')

## Store

In [65]:
edf_copy.to_csv('energy_p3.csv', index=False)

In [76]:
with open('dtypes.pickle', 'wb') as fileobj:
    pickle.dump(edf_copy.dtypes.to_dict(), fileobj, protocol=pickle.HIGHEST_PROTOCOL)